In [89]:
import pandas as pd
# import mysql.connector as mycon
import kagglehub
from kagglehub import KaggleDatasetAdapter
import time

# This pulls in the wildfire data and reads it

In [2]:
use_cols = ['OBJECTID', 'FIRE_NAME', 'DISCOVERY_DATE', 'NWCG_GENERAL_CAUSE', 'FIRE_SIZE', 'STATE', 'FIPS_NAME']

# Download latest version
wildfire_df = kagglehub.load_dataset(handle = "behroozsohrabi/us-wildfire-records-6th-edition", path = "data.csv", 
                                   adapter = KaggleDatasetAdapter.PANDAS, 
                                   pandas_kwargs={"usecols": use_cols, "compression": "zip"})

c:\Users\Taylor\AppData\Local\Programs\Python\Python313\Lib\site-packages\kagglehub\pandas_datasets.py:91: DtypeWarning: Columns (14,38) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


In [3]:
# Convert DISCOVERY_DATE to datetime object to remove records prior to year 2000

wildfire_df['DISCOVERY_DATE'] = pd.to_datetime(wildfire_df['DISCOVERY_DATE'], format = ('%m/%d/%Y'))

wildfire_df = wildfire_df[wildfire_df['DISCOVERY_DATE'].dt.year > 2000]

In [4]:
# Rename some columns for clarity

wildfire_df = wildfire_df.rename(columns = {"NWCG_GENERAL_CAUSE": "SPECIFIC_CAUSE", "FIPS_NAME": "COUNTY"})

In [5]:
# Filter states that we're keeping

states_to_keep = ['CA', 'TX', 'GA', 'FL', 'AZ']

wildfire_df.loc[~wildfire_df['STATE'].isin(states_to_keep), :] = None

wildfire_df.dropna(inplace = True)

In [6]:
# Change to int for compatiabilty with Primary Key

wildfire_df['OBJECTID'] = wildfire_df['OBJECTID'].astype(int)

# This pulls in housing data

In [7]:
bottom_tier_housing = pd.read_parquet('data/bottom_housing.parquet')
top_tier_housing = pd.read_parquet('data/top_housing.parquet')

## Remove Columns we don't use

In [8]:
remove_cols = ['RegionID', 'RegionType', 'StateName', 'Metro', 'SizeRank']

bottom_tier_housing.drop(columns = remove_cols, inplace = True)
top_tier_housing.drop(columns = remove_cols, inplace = True)

## Remove rows with states we aren't using in a way that modifies the original dataframe in-place

In [9]:
states = ['CA', 'TX', 'GA', 'FL', 'AZ']

bottom_tier_housing.loc[~bottom_tier_housing['State'].isin(states), :] = None
top_tier_housing.loc[~top_tier_housing['State'].isin(states), :] = None

bottom_tier_housing.dropna(inplace = True)
top_tier_housing.dropna(inplace = True)


## Combine the data frames and pivot long the dates and prices

In [10]:
housing_df = pd.concat([bottom_tier_housing, top_tier_housing], axis = 0, ignore_index = True)

In [11]:
housing_df = housing_df.melt(id_vars = ['RegionName', 'State', 'CountyName'], var_name = 'Date', value_name = 'Price')
housing_df['Date'] = pd.to_datetime(housing_df['Date']).dt.date

# Rent Data (load and remove columns'state values)

In [12]:
rent_index = pd.read_csv("data/Observed Rent Index by City.csv")

In [13]:
# Columns and state values to remove are same as housing sales data

rent_index.drop(columns = remove_cols, inplace = True)
rent_index.loc[~rent_index['State'].isin(states), :] = None

rent_index.dropna(inplace = True)

## Pivot long the date and price data

In [14]:
rent_index = rent_index.melt(id_vars = ['RegionName', 'State', 'CountyName'], var_name = 'Date', value_name = 'Price')
rent_index['Date'] = pd.to_datetime(housing_df['Date']).dt.date

# Create Location Table

## Pull in location info from Census

In [15]:
ca_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st06_ca_place_by_county2020.txt', delimiter = '|')
tx_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st48_tx_place_by_county2020.txt', delimiter = '|')
ga_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st13_ga_place_by_county2020.txt', delimiter = '|')
fl_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st12_fl_place_by_county2020.txt', delimiter = '|')
az_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st04_az_place_by_county2020.txt', delimiter = '|')

## Drop Columns

In [16]:
columns_to_drop = ['PLACENS', 'TYPE', 'CLASSFP', 'FUNCSTAT']

ca_codes.drop(columns = columns_to_drop, inplace = True)
tx_codes.drop(columns = columns_to_drop, inplace = True)
ga_codes.drop(columns = columns_to_drop, inplace = True)
fl_codes.drop(columns = columns_to_drop, inplace = True)
az_codes.drop(columns = columns_to_drop, inplace = True)

In [17]:
# Combine df to make it easier to load

combined_df = pd.concat([ca_codes, tx_codes, ga_codes, fl_codes, az_codes], axis = 0, ignore_index=True)

# Load Population Data into dictionary

In [18]:
state_population_dict = {}

states = ['CA', 'TX', 'GA', 'FL', 'AZ']

for state in states:
    state_population_dict[state] = pd.read_csv(f"data/{state} City population estimates.csv")

# Sync population and location data

In [19]:
location_dict = {
    'CA': ca_codes,
    'TX': tx_codes,
    'GA': ga_codes,
    'FL': fl_codes,
    'AZ': az_codes
}

In [20]:
def sync(states_list):
    for state in states_list:
        pop_df = state_population_dict[state]
        location_df = location_dict[state]

        columns_to_rename = {'PLACE': 'PLACEFP', 'NAME': 'PLACENAME'}

        pop_df = pop_df.rename(columns = columns_to_rename)

        merged_df = pop_df.merge(
            location_df[['COUNTYFP', 'COUNTYNAME', 'PLACEFP', 'PLACENAME']],
            on = ["PLACEFP", 'PLACENAME'],
            how = 'left'
        )
        merged_df.drop(columns = ['SUMLEV', 'COUSUB', 'CONCIT', 'PRIMGEO_FLAG', 'FUNCSTAT', 'ESTIMATESBASE2010', 'COUNTY'], inplace = True)
        merged_df = merged_df.drop_duplicates()
        merged_df = merged_df.dropna()
        merged_df['COUNTYFP'] = merged_df['COUNTYFP'].astype(int)
        state_population_dict[state] = merged_df

In [21]:
sync(states)

# Map state and county codes to wildfire/housing/rent data

In [22]:
# Mapping only for CA, TX, FL, GA, and AZ

state_fips_mapping = {
    "CA": 6,  # California
    "TX": 48, # Texas
    "FL": 12, # Florida
    "GA": 13, # Georgia
    "AZ": 4   # Arizona
}

wildfire_df['state_id'] = wildfire_df['STATE'].map(state_fips_mapping)
housing_df['state_id'] = housing_df['State'].map(state_fips_mapping)
rent_index['state_id'] = rent_index['State'].map(state_fips_mapping)

In [23]:
# Realign column names to prepare for county code mapping

combined_df.rename(columns = {'COUNTYNAME': 'county_name', 'COUNTYFP': 'county_id', 'STATEFP': 'state_id'}, inplace = True)
rent_index.rename(columns = {'CountyName': 'county_name'}, inplace = True)
housing_df.rename(columns = {'CountyName': 'county_name'}, inplace = True)
wildfire_df.rename(columns = {'COUNTY': 'county_name'}, inplace = True)

In [29]:
import pandas as pd
from rapidfuzz import process, fuzz  # Much faster than thefuzz
from tqdm import tqdm  # Progress bar

def match_county_id(data_df, counties_df, state_col="state_id", county_col="county_name", threshold=85):
    """
    Matches county names in the dataset to official counties using fuzzy matching.

    Optimizations:
    - Uses `rapidfuzz` (faster fuzzy matching)
    - Caches results to avoid redundant computations
    - Precomputes county name choices per state

    Parameters:
    - data_df (pd.DataFrame): DataFrame containing records that need county_id assignment.
    - counties_df (pd.DataFrame): Official Census counties with state_id, county_name, and county_id.
    - state_col (str): Column name for state_id.
    - county_col (str): Column name for county_name.
    - threshold (int): Minimum similarity score to consider a match.

    Returns:
    - pd.DataFrame: Updated DataFrame with county_id assigned.
    """
    
    tqdm.pandas()  # Enable progress bar
    
    # Create lookup dictionary: {state_id: (county_name_list, county_id_map)}
    county_lookup = {
        state_id: (
            counties_df[counties_df[state_col] == state_id]["county_name"].tolist(),
            counties_df[counties_df[state_col] == state_id].set_index("county_name")["county_id"].to_dict()
        )
        for state_id in counties_df[state_col].unique()
    }

    # Cache for already matched county names
    match_cache = {}

    def find_best_match(state_id, county_name):
        if state_id in county_lookup:
            county_names, county_id_map = county_lookup[state_id]

            # Check cache first
            cache_key = (state_id, county_name)
            if cache_key in match_cache:
                return match_cache[cache_key]

            # Fuzzy match
            match, score, _ = process.extractOne(county_name, county_names, scorer=fuzz.token_sort_ratio)
            if score >= threshold:
                county_id = county_id_map[match]
                match_cache[cache_key] = county_id  # Cache result
                return county_id

        return None  # No match found

    # Apply fuzzy matching with caching and progress bar
    data_df["county_id"] = data_df.progress_apply(lambda row: find_best_match(row[state_col], row[county_col]), axis=1)

    return data_df




In [ ]:
rent_index = match_county_id(rent_index, combined_df)
housing_df = match_county_id(housing_df, combined_df)
wildfire_df = match_county_id(wildfire_df, combined_df)
population_df = pd.concat(state_population_dict.values(), ignore_index = True)

  0%|          | 0/35760 [00:00<?, ?it/s]

100%|██████████| 35760/35760 [00:01<00:00, 28064.08it/s]


In [45]:
# Take care of missing values in wildfire dataframe, fill in variables accordingly

def fill_missing_values(df):
    """
    Fill missing values based on the state column.
    """
    df.loc[df['STATE'] == 'FL', 'county_id'] = df.loc[df['STATE'] == 'FL', 'county_id'].fillna(86)
    df.loc[df['STATE'] == 'GA', 'county_id'] = df.loc[df['STATE'] == 'GA', 'county_id'].fillna(29)
    return df

wildfire_df = fill_missing_values(wildfire_df)


In [129]:
population_df['CENSUS2010POP'] = population_df['CENSUS2010POP'].apply(lambda x: 0 if x == 'A' else x)

# Final dataframes

rent_index  
housing_df  
wildfire_df  
combined_df  
population_df

# Load data into schema

In [49]:
import mysql.connector as mysql
from mysql.connector import Error

In [48]:
server_name = 'wildfire-watch1.mysql.database.azure.com'
user = 'tkirk'
port = 3306
password = 'Wildfire507'
database = 'wildfire_housing'
ssl = "/DigiCertGlobalRootCA.crt (1).pem"

In [88]:
# function to reconnect to server in case of timeout error

def reconnect():

    global conn, cursor

    try:
        conn = mysql.connect(
            host = server_name,
            user = user,
            port = port,
            password = password,
            database = database,
            ssl_ca = ssl
        )
        cursor = conn.cursor()
        print("Reconnected")

    except Error as err:
        print(f"❌ Connection unsuccessful {err}")
        time.sleep(3)
        reconnect()

# Load locations data

In [64]:
cursor = conn.cursor()

columns = combined_df.columns

data_to_insert = list(combined_df[columns].itertuples(index = False, name = None))

insert_clause = """
INSERT INTO locations (state_name, state_id, county_id, county_name, place_id, place_name)
Values (%s, %s, %s, %s, %s, %s)
"""

try:
    cursor.executemany(insert_clause, data_to_insert)
    conn.commit()
    print('Data loaded successfully')
except Error as err:
    print(f"Data unable to be loaded: {err}")

Data loaded successfully


# Load wildfire data

In [84]:
cursor = conn.cursor()

columns = wildfire_df.columns

data_to_insert = list(wildfire_df[columns].itertuples(index = False, name = None))

clause = """
INSERT INTO wildfire (fire_id, fire_name, discovery_date, cause, fire_size, state_name, county_name, state_id, county_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

batch_size = 10000
start_index = 140000
try:
    for i in range(start_index, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i+batch_size]
        cursor.executemany(clause, batch)
        conn.commit()
        print('Data loaded successfully')
except Error as err:
    print(f"Data unable to be loaded: {err}")

Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully
Data loaded successfully


# Load housing data

In [ ]:
cursor = conn.cursor()

columns = housing_df.columns

data_to_insert = list(housing_df[columns].itertuples(index = False, name = None))

clause = """
INSERT INTO housing (region_name, state_name, county_name, assessment_date, price, state_id, county_id)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

batch_size = 15000

try:
    for i in range(start_index, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i + batch_size]
        
        try:
            cursor.executemany(clause, batch)
            conn.commit()
            print(f"Data loaded successfully for batch {i // batch_size + 1}")
        except Error as e:
            if e == '2013 (HY000): Lost connection to MySQL server during query':
                print("Lost connection, attempting to reconnect...")
                reconnect()
                cursor.executemany(clause, batch)  # Retry query
                conn.commit()

except Error as err:
    print(f"Error: {err}")
    

Data loaded successfully for batch 17
Data loaded successfully for batch 18
Data loaded successfully for batch 19
Data loaded successfully for batch 20
Data loaded successfully for batch 21
Data loaded successfully for batch 22
Data loaded successfully for batch 23
Data loaded successfully for batch 24
Data loaded successfully for batch 25
Data loaded successfully for batch 26
Data loaded successfully for batch 27
Data loaded successfully for batch 28
Data loaded successfully for batch 29
Data loaded successfully for batch 30
Data loaded successfully for batch 31
Data loaded successfully for batch 32
Data loaded successfully for batch 33
Data loaded successfully for batch 34
Data loaded successfully for batch 35
Data loaded successfully for batch 36
Data loaded successfully for batch 37
Data loaded successfully for batch 38
Data loaded successfully for batch 39
Data loaded successfully for batch 40
Data loaded successfully for batch 41
Data loaded successfully for batch 42
Data loaded 

# Load rental data

In [ ]:
cursor = conn.cursor()

columns = rent_index.columns

data_to_insert = list(rent_index[columns].itertuples(index = False, name = None))

clause = """
INSERT INTO rentals (region_name, state_name, county_name, assessment_date, price, state_id, county_id)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

batch_size = 15000

try:
    for i in range(0, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i+batch_size]

        try:
            cursor.executemany(clause, batch)
            conn.commit()
            print(f"Data loaded successfully for batch {i // batch_size + 1}")
        except Error as e:
            if e == "2013 (HY000): Lost connection to MySQL server during query":
                print("Lost connection, attempting to reconnect...")
                reconnect()
                cursor.executemany(clause, batch)
                conn.commmit()
except Error as err:
    print(err)

Data loaded successfully for batch 1
Data loaded successfully for batch 2
Data loaded successfully for batch 3


# Load population data

In [ ]:
cursor = conn.cursor()

data_to_insert = list(population_df[['STATE', 'COUNTYFP', 'PLACEFP', 'CENSUS2010POP', 'POPESTIMATE2011', 'POPESTIMATE2012', 
                                     'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015','POPESTIMATE2016', 'POPESTIMATE2017', 
                                     'POPESTIMATE2018','POPESTIMATE2019']].itertuples(index = False, name = None))

clause = """
INSERT IGNORE INTO census (state_id, county_id, place_id, true_pop_2010, pop_estimate_2011, pop_estimate_2012, pop_estimate_2013, pop_estimate_2014, 
pop_estimate_2015, pop_estimate_2016, pop_estimate_2017, pop_estimate_2018, pop_estimate_2019)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    for i in range(0, len(data_to_insert)):
        batch = data_to_insert[i:i+batch_size]
        cursor.executemany(clause, batch)
        conn.commit()
        print('Data loaded successfully')
except Error as err:
    print(f"Data unable to be loaded: {err}")

Data loaded successfully
